In [1]:
import pandas as pd 
df = pd.read_csv("clean2.csv")
df

,Unnamed: 0,Unnamed: 0.1,Text,oh_label,Label,user_links_dropped,spelling_corrected,hashtag_topics,expanded,wiki_topics,clean_regex,clean_lower,tokenized_text,remaining_words
0,0,0,`- This is not ``creative``. Those are the di...,0,0,`- This is not ``creative``. Those are the dic...,`- This is not ``creative``. Those are the dic...,[],`- This is not ``creative``. Those are the dic...,[],This is not creative Those are the dictionary...,this is not creative those are the dictionary...,"['this', 'is', 'not', 'creative', 'those', 'ar...","['creative', 'dictionary', 'definitions', 'ter..."
1,1,1,` :: the term ``standard model`` is itself le...,0,0,` :: the term ``standard model`` is itself les...,` :: the term ``standard model`` is itself les...,[],` :: the term ``standard model`` is itself les...,[],the term standard model is itself less NPOV ...,the term standard model is itself less npov ...,"['the', 'term', 'standard', 'model', 'is', 'it...","['term', 'standard', 'model', 'less', 'npov', ..."
2,2,2,"True or false, the situation as of March 200...",0,0,"True or false, the situation as of March 2002 ...",True or false the situation as of March 2002 w...,[],"True or false, the situation as of March 2002 ...",[],True or false the situation as of March 2002 w...,true or false the situation as of march 2002 w...,"['true', 'or', 'false', 'the', 'situation', 'a...","['true', 'false', 'situation', 'march', '2002'..."
3,3,3,"Next, maybe you could work on being less cond...",0,0,"Next, maybe you could work on being less conde...",next maybe you could work on being less condes...,[],"Next, maybe you could work on being less conde...",[],Next maybe you could work on being less condes...,next maybe you could work on being less condes...,"['next', 'maybe', 'you', 'could', 'work', 'on'...","['next', 'maybe', 'could', 'work', 'less', 'co..."
4,4,4,This page will need disambiguation.,0,0,This page will need disambiguation.,This page will need disambiguation.,[],This page will need disambiguation.,[],This page will need disambiguation,this page will need disambiguation,"['this', 'page', 'will', 'need', 'disambiguati...","['page', 'need', 'disambiguation']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204954,205173,205173,228((( real!!!! by walt disney=,0,0,228((( real!!!! by walt disney=,228((( real!!!! by walt disney,[],228((( real!!!! by walt disney=,[],228 real by walt disney,228 real by walt disney,"['228', 'real', 'by', 'walt', 'disney']","['228', 'real', 'walt', 'disney']"
204955,205174,205174,Status-Online Im ZxkillergirlzX! I'm Zxkillerg...,0,0,Status-Online Im ZxkillergirlzX! I'm Zxkillerg...,Status-Online Im ZxkillergirlzX! I'm Zxkillerg...,[],Status-Online I Am ZxkillergirlzX! I am Zxkill...,[],StatusOnline I Am ZxkillergirlzX I am Zxkiller...,statusonline i am zxkillergirlzx i am zxkiller...,"['statusonline', 'i', 'am', 'zxkillergirlzx', ...","['statusonline', 'zxkillergirlzx', 'zxkillergi..."
204956,205175,205175,JR so cute EXO M Better I agree like yeah yeah...,0,0,JR so cute EXO M Better I agree like yeah yeah...,or so cute EXO i Better I agree like yeah yeah...,[],JR so cute EXO M Better I agree like yeah yeah...,[],JR so cute EXO M Better I agree like yeah yeah...,jr so cute exo m better i agree like yeah yeah...,"['jr', 'so', 'cute', 'exo', 'm', 'better', 'i'...","['jr', 'cute', 'exo', 'better', 'agree', 'like..."
204957,205176,205176,! !,0,0,! !,! !,[],! !,[],,,[],[]


In [3]:
df['Label'].value_counts()

0    178410
1     25879
2       670
Name: Label, dtype: int64

In [4]:
# stratified sampling
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(670))

In [5]:
df['Label'].value_counts()

0    670
1    670
2    670
Name: Label, dtype: int64

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Hey|Hi|I|You)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [re.sub(r'[^\w\s]|_', '', s) for s in sentences]
    sentences = [s.lower() for s in sentences]
    sentences = [s.strip() for s in sentences] 
    sentences = list(filter(None, sentences))
    return sentences

df['sentences'] = df['expanded'].apply(split_into_sentences)

In [8]:
def get_sentiment_score(sentence_list):
    paragraph = ". ".join(sentence_list)
    sentiment_score = analyzer.polarity_scores(paragraph)['compound']
    return sentiment_score

df['Sentiment Score (Original Text)'] = df['sentences'].apply(get_sentiment_score)

In [10]:
df['Sentiment Score (PDTB split)'] = 0
df['Weighted Discourse Fragments (PDTB)'] = ''
df['Sentiment Score (Dependency split)'] = 0
df['Weighted Discourse Fragments (Dependency split)'] = ''

In [17]:
with open('contrastive_markers.txt') as f:
    contrastive_markers = f.readlines()
contrastive_markers = [re.sub(r'\n', '', marker) for marker in contrastive_markers]
contrastive_markers.pop(0)
contrastive_markers = [re.sub(r'[^\w\s]|_', '', marker) for marker in contrastive_markers]
contrastive_markers = [marker.strip() for marker in contrastive_markers]
contrastive_markers

['after everything',
 'all the same',
 'alternatively',
 'although',
 'anyroad',
 'anyway',
 'anyhow',
 'anywho',
 'anywise',
 'at any cost',
 'au contraire',
 'be that as it may',
 'be it as it may',
 'besides',
 'but',
 'but for all that',
 'by contrast',
 'come what may',
 'contradictorily',
 'contrastingly',
 'contrarily',
 'contrariwise',
 'despite',
 'despite that',
 'despite this fact',
 'even so',
 'either way',
 'conversely',
 'for all that',
 'furthermore',
 'having said that',
 'howbeit',
 'however',
 'in any case',
 'in any event',
 'in contrast',
 'instead',
 'in spite of',
 'in opposition',
 'inversely',
 'irregardless',
 'just the same',
 'meanwhile',
 'nevertheless',
 'natheless',
 'no matter',
 'no matter what',
 'nonetheless',
 'notwithstanding',
 'on one hand',
 'on the flip side',
 'on the other hand',
 'on the other side of the coin',
 'on the contrary',
 'oppositely',
 'rather',
 'regardless',
 'still',
 'still and all',
 'then again',
 'that said',
 'though',
 't

In [180]:
def discourse_segmentation(sentence_list):
    weighted_sentences = {}
    for segment in sentence_list:
        segment_list = segment.split(" ")
        discourse_found = set(segment_list).intersection(contrastive_markers)
        
        weight = 1
        if len(discourse_found) == 0:
            weighted_sentences[segment] = weight
        else:
            while len(discourse_found) > 0:
                discourse = discourse_found.pop()
                seg = segment.split(discourse)
                
                seg = list(filter(None, seg))    
                seg = [s.strip() for s in seg] 
                
                for i in range(len(seg)):
                    if len(seg) > 1:
                        weighted_sentences[seg[i]] = weight
                        weight = weight * 1.5 
                    else:
                        weighted_sentences[seg[i]] = weight
    return weighted_sentences

df['Weighted Discourse Fragments (PDTB)'] = df['sentences'].apply(discourse_segmentation)

In [167]:
with open('profanities.txt') as f:
    profanities = f.readlines()
profanities = [re.sub(r'\n', '', profanity) for profanity in profanities]
profanities = [re.sub(r'[^\w\s]|_', '', profanity) for profanity in profanities]
profanities = [profanity.strip() for profanity in profanities]
profanities

['suck',
 'sucks',
 'sck',
 'suuk',
 'fk',
 'fuck',
 'fck',
 'bitch',
 'biatch',
 'shit',
 'shitty',
 'sht',
 'shtty',
 'cocky',
 'damn',
 'wtf',
 'jesus']

In [197]:
def get_final_sentiment_score(weighted_dictionary):
    total_score = 0
    
    for key, values in weighted_dictionary.copy().items():
        # find if any profanities and remove them, as well as add weights to profanity 
        word_list = key.split(" ")
        
        if len(set(word_list).intersection(profanities)) > 0:
            # locate the original key of dictionary then update the key and the value to without profanity
            original = " ".join(word_list)
            weighted_dictionary[original] =  weighted_dictionary[original] * 1.5
            word_list.remove(set(word_list).intersection(profanities).pop())
            new_key = " ".join(word_list)
            weighted_dictionary[new_key] = weighted_dictionary.pop(key)
    
    # normalization of all weights such that sum of weights for all sentences will always add up to 1 
    sum_weights = sum(weighted_dictionary.values())
    scaled_weights = {key: ((v) / (sum_weights) )  for (key, v) in weighted_dictionary.items() }

    # sentiment scoring 
    for key, values in scaled_weights.items():
        total_score += analyzer.polarity_scores(key)['compound'] * scaled_weights[key]
    
    return total_score

df['Sentiment Score (PDTB split)'] = df['Weighted Discourse Fragments (PDTB)'].apply(get_final_sentiment_score)

In [200]:
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')

2022-03-31 00:50:57 INFO: Downloading default packages for language: en (English)...
2022-03-31 00:51:00 INFO: File exists: C:\Users\lingl\stanza_resources\en\default.zip
2022-03-31 00:51:12 INFO: Finished downloading models and saved to C:\Users\lingl\stanza_resources.


2022-03-31 00:51:16 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-03-31 00:51:16 INFO: Use device: cpu
2022-03-31 00:51:16 INFO: Loading: tokenize
2022-03-31 00:51:16 INFO: Loading: pos
2022-03-31 00:51:16 INFO: Loading: lemma
2022-03-31 00:51:16 INFO: Loading: depparse
2022-03-31 00:51:17 INFO: Loading: sentiment
2022-03-31 00:51:17 INFO: Loading: constituency
2022-03-31 00:51:20 INFO: Loading: ner
2022-03-31 00:51:21 INFO: Done loading processors!


In [201]:
def get_index(word_id, sentence_list, document):
    for i in range(len(sentence_list)):
        for records in document.sentences[i].dependencies:
            no_str = [word for word in records if not isinstance(word,str)]
            for record in no_str:
                if record.id == word_id:
                    return i

In [202]:
def get_discourse_segments(sentences_list):
    entire_sentence = ".".join(sentences_list)
    doc = nlp(entire_sentence)
    discourse_marker_sentences = []
    for sentence in doc.sentences:
        for records in sentence.dependencies:
            no_str = [word for word in records if not isinstance(word,str)]
            for record in no_str:
                if record.deprel == "mark":
                    discourse_marker_sentence = sentences_list[get_index(record.id, sentences_list, doc)]
                    discourse_parent_index = get_index(record.head, sentences_list, doc)
                    discourse_parent_sentence = sentences_list[discourse_parent_index]
                    if discourse_marker_sentence != discourse_parent_sentence:
                        joined = discourse_marker_sentence + ". " + discourse_parent_sentence
                        if joined not in discourse_marker_sentences:
                            discourse_marker_sentences.append(joined)
                    elif discourse_marker_sentence == discourse_parent_sentence:
                        if discourse_marker_sentence not in discourse_marker_sentences:
                            discourse_marker_sentences.append(discourse_marker_sentence)    
                elif sentences_list[get_index(record.id, sentences_list, doc)] not in discourse_marker_sentences:
                    discourse_marker_sentences.append(sentences_list[get_index(record.id, sentences_list, doc)])
    return discourse_marker_sentences
df['Weighted Discourse Fragments (Dependency split)'] = df['sentences'].apply(get_discourse_segments)

In [209]:
def change_to_dict(sentences_list):
    sentence_dict = {}
    for i in sentences_list:
        sentence_dict[i] = 1
    return sentence_dict

df['Weighted Discourse Fragments (Dependency split)'] = df['Weighted Discourse Fragments (Dependency split)'].apply(change_to_dict)

In [211]:
df['Sentiment Score (Dependency split)'] = df['Weighted Discourse Fragments (Dependency split)'].apply(get_final_sentiment_score)

In [213]:
df.to_csv("discourse_segments_sentiment_scored.csv")